In [107]:
! pip install sunpy[all]
from IPython.display import clear_output
clear_output()

In [108]:
import warnings

warnings.filterwarnings("ignore")

# predicting coronal mass ejections using machine learning methods

A Coronal Mass Ejection (CME) throws magnetic flux and plasma from the Sun into interplanetary space. These eruptions are actually related to solar flares -- in fact, CMEs and solar flares are considered “a single magnetically driven event” ([Webb & Howard 2012](http://adsabs.harvard.edu/abs/2012LRSP....9....3W)), wherein a flare unassociated with a CME is called a confined or compact flare. <br>

In general, the more energetic a flare, the more likely it is to be associated with a CME ([Yashiro et al. 2005](http://adsabs.harvard.edu/abs/2005JGRA..11012S05Y)) -- but this is not, by any means, a rule. For example, [Sun et al. (2015)](http://adsabs.harvard.edu/abs/2015ApJ...804L..28S) found that the largest active region in the last 24 years, shown below, produced 6 X-class flares but not a single observed CME.<br>

In this notebook, we will be predicting whether or not a flaring active region will also emit a CME using a machine learning algorithm from the scikit-learn package called Support Vector Machine.

The analysis that follows is published in [Bobra & Ilonidis, 2016, <i> Astrophysical Journal</i>, 821, 127](http://adsabs.harvard.edu/abs/2016ApJ...821..127B). If you use any of this code, we ask that you cite Bobra & Ilonidis (2016).

To do this analysis, we'll look at every active region observed by the Helioseismic and Magnetic Imager instrument on NASA's Solar Dynamics Observatory (SDO) satellite over the last eight years. Each active region is characterized by a bunch of features. These features describe the magnetic field at the solar surface. One feature, for example, is the total energy contained within an active region. Another is the total flux through an active region. We have 18 features, all of which are calculated every 12 minutes throughout an active region's lifetime. See [Bobra et al., 2014](http://link.springer.com/article/10.1007%2Fs11207-014-0529-3) for more information on how we calculate these features. <br>

We'll then ascribe each active region to one of two classes:

1. The positive class contains flaring active regions that did produce a CME.
2. The negative class contains flaring active regions that did not produce a CME.

First, we'll import some modules.

In [109]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.mlab as mlab
import pandas as pd
import scipy.stats
import requests
import urllib
import json
from datetime import datetime as dt_obj
from datetime import timedelta
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sunpy.time import TimeRange
from sunpy.net import Fido, attrs as a

pd.set_option('display.max_rows', 500)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Now we'll gather the data. The data come from three different places:

1. CME data from SOHO/LASCO and STEREO/SECCHI coronographs, which can be accesed from the [DONKI database](http://kauai.ccmc.gsfc.nasa.gov/DONKI/) at NASA Goddard. This tells us if an active region has produced a CME or not.
2. Flare data from the GOES flare catalog at NOAA, which can be accessed with the `sunpy.instr.goes.get_event_list()` function. This tells us if an active region produced a flare or not.
3. Active region data from the Solar Dynamics Observatory's Heliosesmic and Magnetic Imager instrument, which can be accessed from the [JSOC database](http://jsoc.stanford.edu/) via a JSON API. This gives us the features characterizing each active region.

### step 1: gathering data for the positive class

Let's first query the [DONKI database](http://kauai.ccmc.gsfc.nasa.gov/DONKI/) to get the data associated with the positive class. Be forewarned: there's a lot of data cleaning involved with building the positive class.

In [110]:
# request the data
baseurl = "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR?"
t_start = "2010-05-01"
t_end = "2024-12-31"
url = baseurl+"startDate="+t_start+"&endDate="+t_end

# if there's no response at this time, print warning
response = requests.get(url)
if response.status_code != 200:
    print('cannot successfully get an http response')

In [111]:
# read the data

print("Getting data from", url)
df = pd.read_json(url)

# select flares associated with a linked event (SEP or CME), and
# select only M or X-class flares
events_list = df.loc[df['classType'].str.contains("M|X") & ~df['linkedEvents'].isnull()]

# drop all rows that don't satisfy the above conditions
events_list = events_list.reset_index(drop=True)

Getting data from https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR?startDate=2010-05-01&endDate=2024-12-31


In [112]:
# drop the rows that aren't linked to CME events
for i in range(events_list.shape[0]):
    value = events_list.loc[i]['linkedEvents'][0]['activityID']
    if not "CME" in value:
        print(value, "not a CME, dropping row")
        events_list = events_list.drop([i])
events_list = events_list.reset_index(drop=True)

2011-08-09T08:40:00-SEP-001 not a CME, dropping row
2011-09-07T06:00:00-SEP-001 not a CME, dropping row
2011-09-24T20:45:00-SEP-001 not a CME, dropping row
2013-06-21T06:09:00-SEP-001 not a CME, dropping row
2015-03-06T19:20:00-SEP-001 not a CME, dropping row
2015-06-18T09:25:00-SEP-001 not a CME, dropping row
2015-06-21T20:35:00-SEP-001 not a CME, dropping row
2015-06-21T20:35:00-SEP-001 not a CME, dropping row
2023-07-18T01:00:00-SEP-002 not a CME, dropping row
2023-08-05T10:00:00-SEP-001 not a CME, dropping row
2024-01-22T13:00:00-SEP-001 not a CME, dropping row
2024-05-10T14:50:00-SEP-001 not a CME, dropping row
2024-09-09T20:55:00-SEP-002 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row


Convert the `peakTime` column in the `events_list` dataframe from a string into a datetime object:

In [113]:
def parse_tai_string(tstr):
    year = int(tstr[:4])
    month = int(tstr[5:7])
    day = int(tstr[8:10])
    hour = int(tstr[11:13])
    minute = int(tstr[14:16])
    return dt_obj(year, month, day, hour, minute)


for i in range(events_list.shape[0]):
    events_list['peakTime'].iloc[i] = parse_tai_string(events_list['peakTime'].iloc[i])

Check for Case 1: In this case, the CME and flare exist but NOAA active region number does not exist in the DONKI database.

In [114]:
# Case 1: CME and Flare exist but NOAA active region number does not exist in DONKI database

number_of_donki_mistakes = 0  # count the number of DONKI mistakes
# create an empty array to hold row numbers to drop at the end
event_list_drops = []

for i in range(events_list.shape[0]):
    if (np.isnan(events_list.loc[i]['activeRegionNum'])):
        time = events_list['peakTime'].iloc[i]
        time_range = TimeRange(time, time)
        listofresult = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.FL.GOESCls >= "M1.0",a.hek.OBS.Observatory == "GOES")

        if len(listofresult["hek"]) == 0:
            print(events_list.loc[i]['classType'], "has no match in the GOES flare database ; dropping row.")
            event_list_drops.append(i)
            number_of_donki_mistakes += 1
            continue
        else:
            if (listofresult[0]['ar_noaanum'] == 0):
                print(events_list.loc[i]['activeRegionNum'], events_list.loc[i]
                    ['classType'], "has no match in the GOES flare database ; dropping row.")
                event_list_drops.append(i)
                number_of_donki_mistakes += 1
                continue
            else:
                print("Missing NOAA number:", events_list['activeRegionNum'].iloc[i], events_list['classType'].iloc[i],
                    events_list['peakTime'].iloc[i], "should be", listofresult[0]['ar_noaanum'][0], "; changing now.")
                events_list['activeRegionNum'].iloc[i] = listofresult[0]['ar_noaanum']
                number_of_donki_mistakes += 1

# Drop the rows for which there is no active region number in both the DONKI and GOES flare databases
events_list = events_list.drop(event_list_drops)
events_list = events_list.reset_index(drop=True)
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Missing NOAA number: nan X1.4 2011-09-22 11:01:00 should be 11302 ; changing now.
Missing NOAA number: nan X1.3 2012-03-07 01:14:00 should be 11430 ; changing now.
Missing NOAA number: nan M6.3 2012-03-09 03:53:00 should be 11429 ; changing now.
Missing NOAA number: nan M5.1 2012-05-17 01:47:00 should be 11476 ; changing now.
Missing NOAA number: nan X1.1 2012-07-06 23:08:00 should be 11515 ; changing now.
Missing NOAA number: nan M6.2 2012-07-28 20:56:00 should be 11532 ; changing now.
Missing NOAA number: nan M1.7 2012-11-08 02:23:00 should be 11611 ; changing now.
Missing NOAA number: nan M1.2 2013-03-15 06:58:00 should be 11692 ; changing now.
Missing NOAA number: nan X1.6 2013-05-13 02:17:00 should be 11748 ; changing now.
Missing NOAA number: nan X2.8 2013-05-13 16:05:00 should be 11748 ; changing now.
Missing NOAA number: nan X3.2 2013-05-14 01:11:00 should be 11748 ; changing now.
Missing NOAA number: nan X1.2 2013-05-15 01:48:00 should be 11748 ; changing now.
Missing NOAA num

Now we grab all the data from the GOES database in preparation for checking Cases 2 and 3.

In [115]:
# Grab all the data from the GOES database
t_start = "2010-05-01"
t_end = "2024-12-31"
time_range = TimeRange(t_start, t_end)
listofresults = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.FL.GOESCls >= "M1.0",a.hek.OBS.Observatory == "GOES")
print('Grabbed all the GOES data; there are', len(listofresults["hek"]), 'events.')

Grabbed all the GOES data; there are 2302 events.


Check for Case 2: In this case, the NOAA active region number is wrong in the DONKI database.

In [116]:
# Case 2: NOAA active region number is wrong in DONKI database

# collect all the peak flares times in the NOAA database
peak_times_noaa = [item["event_peaktime"] for item in listofresults["hek"]]

for i in range(events_list.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = events_list['peakTime'].iloc[i]
    if peak_time_donki in peak_times_noaa:
        index = peak_times_noaa.index(peak_time_donki)
    else:
        continue
    # ignore NOAA active region numbers equal to zero
    if (listofresults["hek"][index]['ar_noaanum'] == 0):
        continue
    # if yes, check if the DONKI and NOAA active region numbers match up for this peak time
    # if they don't, flag this peak time and replace the DONKI number with the NOAA number
    if (listofresults["hek"][index]['ar_noaanum'] != int(events_list['activeRegionNum'].iloc[i])):
        print('Messed up NOAA number:', int(events_list['activeRegionNum'].iloc[i]), events_list['classType'].iloc[i],
              events_list['peakTime'].iloc[i], "should be", listofresults["hek"][index]['ar_noaanum'], "; changing now.")
        events_list['activeRegionNum'].iloc[i] = listofresults["hek"][index]['ar_noaanum']
        number_of_donki_mistakes += 1
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up NOAA number: 11943 X1.2 2014-01-07 18:32:00 should be 11944 ; changing now.
Messed up NOAA number: 12051 M1.2 2014-05-07 16:29:00 should be 12055 ; changing now.
Messed up NOAA number: 12160 M1.4 2014-07-01 11:23:00 should be 12106 ; changing now.
Messed up NOAA number: 12282 M2.4 2015-02-09 23:35:00 should be 12280 ; changing now.
Messed up NOAA number: 12321 M1.1 2015-04-23 10:07:00 should be 12322 ; changing now.
Messed up NOAA number: 12565 M7.6 2016-07-23 05:16:00 should be 12567 ; changing now.
Messed up NOAA number: 12565 M5.5 2016-07-23 05:31:00 should be 12567 ; changing now.
Messed up NOAA number: 13191 M6.0 2023-01-15 03:42:00 should be 13188 ; changing now.
Messed up NOAA number: 13312 M1.9 2023-05-22 13:37:00 should be 13314 ; changing now.
Messed up NOAA number: 13561 M1.0 2024-01-23 13:08:00 should be 13559 ; changing now.
Messed up NOAA number: 13615 M9.4 2024-03-30 21:16:00 should be 13515 ; changing now.
Messed up NOAA number: 13638 M1.0 2024-04-19 13:06:00 

Check for Case 3: In this case, the flare peak time is wrong in the DONKI database.

In [117]:
# Case 3: The flare peak time is wrong in the DONKI database.

# create an empty array to hold row numbers to drop at the end
event_list_drops = []

active_region_numbers_noaa = [item["ar_noaanum"]
                              for item in listofresults["hek"]]
flare_classes_noaa = [item["fl_goescls"] for item in listofresults["hek"]]

for i in range(events_list.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = events_list['peakTime'].iloc[i]
    if not peak_time_donki in peak_times_noaa:
        active_region_number_donki = int(
            events_list['activeRegionNum'].iloc[i])
        flare_class_donki = events_list['classType'].iloc[i]
        flare_class_indices = [i for i, x in enumerate(
            flare_classes_noaa) if x == flare_class_donki]
        active_region_indices = [i for i, x in enumerate(
            active_region_numbers_noaa) if x == active_region_number_donki]
        common_indices = list(
            set(flare_class_indices).intersection(active_region_indices))
        if common_indices:
            print("Messed up time:", int(events_list['activeRegionNum'].iloc[i]), events_list['classType'].iloc[i],
                  events_list['peakTime'].iloc[i], "should be", peak_times_noaa[common_indices[0]], "; changing now.")
            events_list['peakTime'].iloc[i] = peak_times_noaa[common_indices[0]]
            number_of_donki_mistakes += 1
        if not common_indices:
            print("DONKI flare peak time",
                  events_list['peakTime'].iloc[i], "has no match; dropping row.")
            event_list_drops.append(i)
            number_of_donki_mistakes += 1

# Drop the rows for which the NOAA active region number and flare class associated with
# the messed-up flare peak time in the DONKI database has no match in the GOES flare database
events_list = events_list.drop(event_list_drops)
events_list = events_list.reset_index(drop=True)

# Create a list of corrected flare peak times
peak_times_donki = [events_list['peakTime'].iloc[i]
                    for i in range(events_list.shape[0])]

print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up time: 11429 X1.1 2012-03-05 04:05:00 should be 2012-03-05 04:09:00.000 ; changing now.
DONKI flare peak time 2012-03-10 17:27:00 has no match; dropping row.
Messed up time: 11745 M5.0 2013-05-22 13:38:00 should be 2013-05-22 13:32:00.000 ; changing now.
DONKI flare peak time 2014-02-09 16:14:00 has no match; dropping row.
Messed up time: 12127 M1.5 2014-08-01 18:12:00 should be 2014-08-01 18:13:00.000 ; changing now.
Messed up time: 12146 M2.0 2014-08-25 15:10:00 should be 2014-08-25 15:11:00.000 ; changing now.
DONKI flare peak time 2014-09-03 13:53:00 has no match; dropping row.
DONKI flare peak time 2014-09-09 00:28:00 has no match; dropping row.
Messed up time: 12172 M2.3 2014-09-23 23:15:00 should be 2014-09-23 23:16:00.000 ; changing now.
Messed up time: 12242 X1.8 2014-12-20 00:24:00 should be 2014-12-20 00:28:00.000 ; changing now.
Messed up time: 12445 M1.9 2015-11-04 03:25:00 should be 2015-11-04 03:26:00.000 ; changing now.
DONKI flare peak time 2021-11-02 02:51:00

This is our final table of events that fall into the positive class:

In [118]:
events_list

,flrID,catalog,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,linkedEvents,sentNotifications
0,2011-02-15T01:44:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-02-15T01:44Z,2011-02-15 01:56:00,2011-02-15T02:06Z,X2.2,S20W10,11158.0,,2015-07-16T19:24Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-02-15T02:25:00-CME-001'}],"[{'messageID': '20110215-AL-001', 'messageIssu..."
1,2011-02-24T07:23:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-02-24T07:23Z,2011-02-24 07:35:00,2011-02-24T07:42Z,M3.5,N14E87,11163.0,,2015-07-16T19:28Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-02-24T08:00:00-CME-001'}],"[{'messageID': '20110224-AL-001', 'messageIssu..."
2,2011-03-07T13:44:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-07T13:44Z,2011-03-07 14:30:00,2011-03-07T15:08Z,M2.0,N11E21,11166.0,,2013-07-18T13:28Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-07T14:40:00-CME-001'}],"[{'messageID': '20110307-AL-001', 'messageIssu..."
3,2011-03-07T19:43:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-07T19:43Z,2011-03-07 20:12:00,2011-03-07T21:40Z,M3.7,N30W48,11164.0,,2013-07-18T18:41Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-07T20:12:00-CME-001'}],"[{'messageID': '20110307-AL-002', 'messageIssu..."
4,2011-03-08T03:37:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-08T03:37Z,2011-03-08 03:58:00,2011-03-08T04:20Z,M1.5,S21E72,11171.0,,2013-07-18T19:11Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-08T05:00:00-CME-001'}],"[{'messageID': '20110308-AL-004', 'messageIssu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,2024-12-25T04:46:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-S: EXIS 1.0-8.0'}],2024-12-25T04:46Z,2024-12-25 04:49:00,2024-12-25T04:53Z,M4.9,S18E10,13932.0,,2024-12-25T13:40Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-25T04:48:00-CME-001'}],None
519,2024-12-25T23:59:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-25T23:59Z,2024-12-26 00:30:00,2024-12-26T00:53Z,M3.1,N20E52,13938.0,Simultaneous flaring from AR 3938 (N20E52) and...,2024-12-26T16:51Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-26T01:25:00-CME-001'}],None
520,2024-12-29T04:18:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-29T04:18Z,2024-12-29 04:30:00,2024-12-29T04:45Z,M2.0,S15E30,13939.0,,2024-12-29T14:21Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-29T06:24:00-CME-001'}],None
521,2024-12-29T16:56:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-29T16:56Z,2024-12-29 17:08:00,2024-12-29T17:20Z,M3.3,S12E23,13939.0,,2024-12-29T20:43Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-29T18:24:00-CME-001'}],None


Now let's query the JSOC database to see if there are active region parameters at the time of the flare. First read the following file to map NOAA active region numbers to HARPNUMs (a HARP, or an HMI Active Region Patch, is the preferred numbering system for the HMI active regions as they appear in the magnetic field data before NOAA observes them in white light):

In [119]:
answer = pd.read_csv('http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt', sep=' ')

Now, let's determine at which time we'd like to predict CMEs. In general, many people try to predict a CME either 24 or 48 hours before it happens. We can report both in this study by setting a variable called `timedelayvariable`:

In [32]:
def get_the_jsoc_data(event_count, t_rec):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """

    catalog_data = []
    classification = []

    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = answer[answer['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        # construct jsoc_info queries and query jsoc database; we are querying for 25 keywords
        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue

        # read the JSON output
        data = response.json()

        # if there are no data at this time, quit
        if data['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # latitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)

        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: failed to find CEA JSOC data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # read the JSON output
        latitude_information = response.json()

        # if there are no data at this time, quit
        if latitude_information['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
        CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
        if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
            print('skip: latitude is out of range for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        if ('MISSING' in str(data['keywords'])):
            print('skip: there are some missing keywords for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', idx.HARPNUM.values[0], 'at time', t_rec[i])

        individual_flare_data = []
        for j in range(18):
            individual_flare_data.append(
                float(data['keywords'][j]['values'][0]))

        catalog_data.append(list(individual_flare_data))

        single_class_instance = [idx.HARPNUM.values[0], str(
            int(listofactiveregions[i])), listofgoesclasses[i], t_rec[i]]
        classification.append(single_class_instance)

    return catalog_data, classification

Now we prepare the data to be fed into the function:

In [17]:
listofactiveregions = list(events_list['activeRegionNum'].values.flatten())
listofgoesclasses = list(events_list['classType'].values.flatten())

And call the function:

In [33]:
for i in [8,12]:
    timedelayvariable = i
    t_rec = [(events_list['peakTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(events_list.shape[0])]
    positive_result = get_the_jsoc_data(events_list.shape[0], t_rec)
    positive_df = pd.concat([pd.DataFrame(positive_result[0]), pd.DataFrame(positive_result[1])], axis=1)
    positive_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]
    positive_df.to_csv(f'positive_df_{timedelayvariable}.csv', index=False)

===== 0 =====
skip: there are no data for HARPNUM 377 at time 2011.02.14_17:56_TAI
===== 1 =====
skip: there are no data for HARPNUM 392 at time 2011.02.23_23:35_TAI
===== 2 =====
accept NOAA Active Region number 11166 and HARPNUM 401 at time 2011.03.07_06:30_TAI
===== 3 =====
accept NOAA Active Region number 11164 and HARPNUM 393 at time 2011.03.07_12:12_TAI
===== 4 =====
skip: there are no data for HARPNUM 415 at time 2011.03.07_19:58_TAI
===== 5 =====
accept NOAA Active Region number 11226 and HARPNUM 637 at time 2011.06.06_22:41_TAI
===== 6 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.08.03_05:48_TAI
===== 7 =====
skip: there are no data for HARPNUM 750 at time 2011.08.03_19:57_TAI
===== 8 =====
accept NOAA Active Region number 11283 and HARPNUM 833 at time 2011.09.07_14:38_TAI
===== 9 =====
skip: latitude is out of range for HARPNUM 892 at time 2011.09.22_03:01_TAI
===== 10 =====
accept NOAA Active Region number 11302 and HARPNUM 892 at time 2011.09.24

### step 2: gathering data for the negative class

To gather the examples for the negative class, we only need to:

1. Query the GOES database for all the M- and X-class flares during our time of interest, and
2. Select the ones that are not associated with a CME.

In [22]:
# select peak times that belong to both classes
all_peak_times = np.array([(listofresults["hek"][i]['event_peaktime'])
                           for i in range(len(listofresults["hek"]))])

negative_class_possibilities = []
counter_positive = 0
counter_negative = 0
for i in range(len(listofresults["hek"])):
    this_peak_time = all_peak_times[i]
    if (this_peak_time in peak_times_donki):
        counter_positive += 1
    else:
        counter_negative += 1
        this_instance = [listofresults["hek"][i]['ar_noaanum'],
                         listofresults["hek"][i]['fl_goescls'], listofresults["hek"][i]['event_peaktime']]
        negative_class_possibilities.append(this_instance)
print("There are", counter_positive, "events in the positive class.")
print("There are", counter_negative, "events in the negative class.")

There are 534 events in the positive class.
There are 1768 events in the negative class.


Again, we compute times that are one day before the flare peak time and convert it into a string that JSOC can understand:

And again, we query the JSOC database to see if these data are present:

In [23]:
listofactiveregions = list(negative_class_possibilities[i][0] for i in range(counter_negative))
listofgoesclasses = list(negative_class_possibilities[i][1] for i in range(counter_negative))

In [24]:
for i in [8,12,24,36,48]:
    timedelayvariable = i
    t_rec = np.array([(negative_class_possibilities[i][2] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(len(negative_class_possibilities))])
    negative_result = get_the_jsoc_data(counter_negative, t_rec)
    negative_df = pd.concat([pd.DataFrame(negative_result[0]), pd.DataFrame(negative_result[1])], axis=1)
    negative_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]
    negative_df.to_csv(f'negative_df_{timedelayvariable}.csv', index=False)

===== 0 =====
accept NOAA Active Region number 11069 and HARPNUM 8 at time 2010.05.05_09:19_TAI
===== 1 =====
accept NOAA Active Region number 11081 and HARPNUM 54 at time 2010.06.11_16:57_TAI
===== 2 =====
skip: latitude is out of range for HARPNUM 49 at time 2010.06.12_21:39_TAI
===== 3 =====
accept NOAA Active Region number 11093 and HARPNUM 115 at time 2010.08.07_10:24_TAI
===== 4 =====
accept NOAA Active Region number 11112 and HARPNUM 211 at time 2010.10.16_11:12_TAI
===== 5 =====
skip: there are no data for HARPNUM 245 at time 2010.11.04_15:58_TAI
===== 6 =====
skip: there are no data for HARPNUM 245 at time 2010.11.04_15:58_TAI
===== 7 =====
skip: there are no data for HARPNUM 245 at time 2010.11.05_05:29_TAI
===== 8 =====
skip: there are no data for HARPNUM 245 at time 2010.11.06_07:36_TAI
===== 9 =====
skip: there are no data for HARPNUM 345 at time 2011.01.27_17:03_TAI
===== 10 =====
accept NOAA Active Region number 11153 and HARPNUM 362 at time 2011.02.08_17:31_TAI
===== 11

## Temporal data

In [120]:
def get_the_jsoc_data(event_count, t_rec, t_rec1):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """

    catalog_data = []
    classification = []

    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = answer[answer['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        # construct jsoc_info queries and query jsoc database; we are querying for 25 keywords
        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response = requests.get(url)
        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue
        # read the JSON output
        data = response.json()
        # if there are no data at this time, quit
        if data['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        url1 = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec1[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response1 = requests.get(url1)
        # if there's no response at this time, quit
        if response1.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue
        # read the JSON output
        data1 = response1.json()

        # if there are no data at this time, quit
        if data1['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec1[i])
            continue

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # latitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)

        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: failed to find CEA JSOC data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # read the JSON output
        latitude_information = response.json()

        # if there are no data at this time, quit
        if latitude_information['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
        CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
        if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
            print('skip: latitude is out of range for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        if ('MISSING' in str(data['keywords'])):
            print('skip: there are some missing keywords for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', idx.HARPNUM.values[0], 'at time', t_rec[i])
        
        individual_flare_data = []
        for j in range(18):
            if data['keywords'][j]['values'][0] == 'MISSING' or data1['keywords'][j]['values'][0] == 'MISSING':
                individual_flare_data.append(np.nan)
            else:
                individual_flare_data.append(
                float(data['keywords'][j]['values'][0]) - float(data1['keywords'][j]['values'][0]))
        
        catalog_data.append(list(individual_flare_data))

        single_class_instance = [idx.HARPNUM.values[0], str(
            int(listofactiveregions[i])), listofgoesclasses[i], t_rec[i]]
        classification.append(single_class_instance)

    return catalog_data, classification

In [100]:
listofactiveregions = list(events_list['activeRegionNum'].values.flatten())
listofgoesclasses = list(events_list['classType'].values.flatten())

In [101]:
for i in [8,12,24,36,48]:
    timedelayvariable = i
    t_rec = [(events_list['peakTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(events_list.shape[0])]
    t_rec1 = [(events_list['peakTime'].iloc[i] - timedelta(hours=timedelayvariable + 12)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(events_list.shape[0])]
    
    positive_result = get_the_jsoc_data(events_list.shape[0], t_rec, t_rec1)
    positive_df = pd.concat([pd.DataFrame(positive_result[0]), pd.DataFrame(positive_result[1])], axis=1)
    positive_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]
    positive_df.to_csv(f'positive_df_temp_{timedelayvariable}.csv', index=False)

===== 0 =====
skip: there are no data for HARPNUM 377 at time 2011.02.14_17:56_TAI
===== 1 =====
skip: there are no data for HARPNUM 392 at time 2011.02.23_23:35_TAI
===== 2 =====
accept NOAA Active Region number 11166 and HARPNUM 401 at time 2011.03.07_06:30_TAI
===== 3 =====
accept NOAA Active Region number 11164 and HARPNUM 393 at time 2011.03.07_12:12_TAI
===== 4 =====
skip: there are no data for HARPNUM 415 at time 2011.03.07_19:58_TAI
===== 5 =====
accept NOAA Active Region number 11226 and HARPNUM 637 at time 2011.06.06_22:41_TAI
===== 6 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.08.03_05:48_TAI
===== 7 =====
skip: there are no data for HARPNUM 750 at time 2011.08.03_19:57_TAI
===== 8 =====
accept NOAA Active Region number 11283 and HARPNUM 833 at time 2011.09.07_14:38_TAI
===== 9 =====
skip: latitude is out of range for HARPNUM 892 at time 2011.09.22_03:01_TAI
===== 10 =====
accept NOAA Active Region number 11302 and HARPNUM 892 at time 2011.09.24

In [121]:
# select peak times that belong to both classes
all_peak_times = np.array([(listofresults["hek"][i]['event_peaktime'])
                           for i in range(len(listofresults["hek"]))])

negative_class_possibilities = []
counter_positive = 0
counter_negative = 0
for i in range(len(listofresults["hek"])):
    this_peak_time = all_peak_times[i]
    if (this_peak_time in peak_times_donki):
        counter_positive += 1
    else:
        counter_negative += 1
        this_instance = [listofresults["hek"][i]['ar_noaanum'],
                         listofresults["hek"][i]['fl_goescls'], listofresults["hek"][i]['event_peaktime']]
        negative_class_possibilities.append(this_instance)
print("There are", counter_positive, "events in the positive class.")
print("There are", counter_negative, "events in the negative class.")

There are 534 events in the positive class.
There are 1768 events in the negative class.


In [122]:
listofactiveregions = list(negative_class_possibilities[i][0] for i in range(counter_negative))
listofgoesclasses = list(negative_class_possibilities[i][1] for i in range(counter_negative))

In [123]:
for i in [24,36,48]:
    timedelayvariable = i
    t_rec = np.array([(negative_class_possibilities[i][2] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(len(negative_class_possibilities))])
    t_rec1 = np.array([(negative_class_possibilities[i][2] - timedelta(hours=timedelayvariable + 12)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(len(negative_class_possibilities))])
    negative_result = get_the_jsoc_data(counter_negative, t_rec, t_rec1)
    negative_df = pd.concat([pd.DataFrame(negative_result[0]), pd.DataFrame(negative_result[1])], axis=1)
    negative_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]
    negative_df.to_csv(f'negative_df_temp_{timedelayvariable}.csv', index=False)

===== 0 =====
accept NOAA Active Region number 11069 and HARPNUM 8 at time 2010.05.04_17:19_TAI
===== 1 =====
skip: there are no data for HARPNUM 54 at time 2010.06.11_00:57_TAI
===== 2 =====
skip: there are no data for HARPNUM 49 at time 2010.06.11_17:39_TAI
===== 3 =====
accept NOAA Active Region number 11093 and HARPNUM 115 at time 2010.08.06_18:24_TAI
===== 4 =====
accept NOAA Active Region number 11112 and HARPNUM 211 at time 2010.10.15_19:12_TAI
===== 5 =====
skip: there are no data for HARPNUM 245 at time 2010.11.03_11:58_TAI
===== 6 =====
skip: there are no data for HARPNUM 245 at time 2010.11.03_11:58_TAI
===== 7 =====
skip: latitude is out of range for HARPNUM 245 at time 2010.11.04_13:29_TAI
===== 8 =====
skip: latitude is out of range for HARPNUM 245 at time 2010.11.05_15:36_TAI
===== 9 =====
accept NOAA Active Region number 11149 and HARPNUM 345 at time 2011.01.27_01:03_TAI
===== 10 =====
accept NOAA Active Region number 11153 and HARPNUM 362 at time 2011.02.08_01:31_TAI
=

In [10]:
import pandas as pd
data = pd.read_csv("G:/My Drive/Latest Research/08-12-2025/Data/Non_temporal/negative_df_8.csv")
data = data.iloc[:,:-3]
data

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM
0,8.879183e+21,106.259,-0.028766,7646.391,21.287,909.625,59.152,-0.059370,39.680,114.549,-0.005868,1.396328e+13,1.162768e+13,1.229334e+23,32.328,271.359863,4.238,348.242,8
1,4.685040e+21,124.228,-0.001464,4215.686,25.461,461.527,67.939,-0.003954,44.976,126.705,-0.364204,9.934509e+12,1.530912e+12,4.340609e+22,36.194,150.230560,3.678,11.351,54
2,1.292189e+22,94.130,-0.005534,7788.253,25.598,881.468,42.985,-0.015151,40.578,94.000,-0.126395,1.835141e+13,1.463005e+12,2.212192e+23,35.404,537.324768,3.565,118.354,115
3,1.244174e+22,120.401,0.004909,3554.413,17.759,1018.763,58.290,0.016626,38.027,120.840,0.382216,2.097954e+13,7.032378e+12,9.856232e+22,30.805,662.910645,3.722,102.490,211
4,2.112975e+22,80.277,-0.006233,7369.953,20.326,1304.346,44.259,-0.011038,37.316,93.655,-0.200768,2.034143e+13,5.819701e+12,2.513559e+23,31.239,680.607483,4.396,160.072,362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,6.403292e+22,111.839,-0.013456,11295.500,25.174,6154.266,61.175,-0.023133,39.925,124.510,0.093669,8.908738e+13,2.759705e+13,1.154411e+24,34.311,3489.907471,5.108,1035.541,12471
945,6.280158e+22,112.377,-0.013104,11625.480,26.420,5986.431,61.741,-0.022689,40.835,125.160,0.042566,8.766627e+13,2.333112e+13,1.173437e+24,35.129,3500.982666,5.133,995.989,12471
946,6.296255e+22,112.638,-0.012799,11506.890,25.966,5959.842,62.151,-0.022109,40.592,125.547,0.112423,8.637505e+13,2.297799e+13,1.158441e+24,34.869,3486.629150,5.106,970.212,12471
947,6.266328e+22,112.246,-0.013246,11511.890,26.264,5866.003,61.685,-0.022957,40.751,125.169,0.062955,8.553706e+13,2.373881e+13,1.157569e+24,34.987,3481.480469,5.123,1002.888,12471
